### Langchain
[LangChain](https://python.langchain.com/en/latest/index.html) is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model, but will also be:
- Data-aware: connect a language model to other sources of data
- Agentic: allow a language model to interact with its environment

The LangChain framework is designed around these principles.

We will use Langchain framework for rest of the workshop.

#### Question Answering over the docs/index
Question answering in this context refers to question answering over your document data.  For question answering over many documents, you almost always want to create an index over the data. This can be used to smartly access the most relevant documents for a given question, allowing you to avoid having to pass all the documents to the LLM (saving you time and money).

#### Set Environment Variables

In [19]:
import json  
import openai
from Utilities.envVars import *
import os

os.environ["AZURESEARCH_FIELDS_ID"] = "id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "contentVector"
os.environ["AZURESEARCH_FIELDS_TAG"] = "{}"

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openAiEndPoint = f"{OpenAiEndPoint}"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"

#### Generate answer for a question from the document we already indexed in Vector Store

In [20]:
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
        openai.api_type = "azure"
        openai.api_key = OpenAiKey
        openai.api_version = OpenAiVersion
        openai.api_base = f"{OpenAiEndPoint}"

        llm = AzureChatOpenAI(
                openai_api_base=openai.api_base,
                openai_api_version=OpenAiVersion,
                deployment_name=OpenAiChat,
                temperature=temperature,
                openai_api_key=OpenAiKey,
                openai_api_type="azure",
                max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(engine=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
        logging.info("LLM Setup done")
elif embeddingModelType == "openai":
        openai.api_type = "open_ai"
        openai.api_base = "https://api.openai.com/v1"
        openai.api_version = '2020-11-07' 
        openai.api_key = OpenAiApiKey
        llm = ChatOpenAI(temperature=temperature,
        openai_api_key=OpenAiApiKey,
        model_name="gpt-3.5-turbo",
        max_tokens=tokenLength)
        embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

c:\Users\astalati\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(


In [21]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="../Workshop/Data/CSV/Compliance Inbox.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["Question", "Other ways to ask", "Response", "GRI Resource Link"],
    },
    encoding="utf-8",
)
docs = loader.load()

In [22]:
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [23]:
from langchain.chains import RetrievalQA  

retriever = db.as_retriever()
chain = RetrievalQA.from_chain_type(llm = llm, retriever = retriever, verbose=False, chain_type = "stuff")

In [34]:
def runChain(question):
    response = chain.run(question)
    return response

In [35]:
questions = [
    "Can I share my credit report?",
    "Recast Request?",
    "How do I make lock transfer request?",
    "What is this new 3 day wait requirement?",
    "How do I verify if I failed my compliance training?",
    "Do I need to sign commitment letter 3 days prior to closing for NJ?",
    "Will I be able to add co-borrower after ICD?"
    "Give me details on Taxpayer Consent Form",
    "How do I collect fees after my loan was withrawn?",
    "What are the program guidelines for",
    "What are the instructions for to get POA approved?",
    "Texas 50(f)(2) Disclosure",
    "How do I move a file back?",
    "Who can I give a gift to?",
]

for question in questions:
    response = runChain(question)
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("")

Question: Can I share my credit report?
Answer: GRI is not permitted to share the credit report with you under our agreement with the Credit Reporting Agencies. However, you are eligible to receive a free copy of your credit report annually by visiting https://www.annualcreditreport.com/index.action.

Question: Recast Request?
Answer: To request a recast, please direct the borrower to http://www.rate.com/servicing to contact their servicer.

Question: How do I make lock transfer request?
Answer: You can submit a lock transfer request by using the link provided below. The commitment desk should be able to assist you with any questions you have about the lock transfer.

GRI Resource Link: https://www.grintranet.com/NewIntranet/Protected/PageLoader.aspx?pageId=1717

Question: What is this new 3 day wait requirement?
Answer: The new 3 day wait requirement is only applicable when there is a change in the Loan Type or Program (such as from 30 year to 20 year, Conv to FHA, or Fixed to ARM), c